**headers**

In [1]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [2]:
#import packages
import pandas as pd
import requests
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder 
from nba_api.stats.endpoints import boxscoretraditionalv2
from datetime import datetime, timedelta


In [ ]:
# Retrieve all games for a specified season
season = '2022-23'
league_game_finder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
games_df = league_game_finder.get_data_frames()[0]



In [ ]:
# Get a list of all team IDs from the games dataframe
team_ids = games_df['TEAM_ID'].unique()

# Initialize an empty list to store box scores for each team
team_box_scores = []


In [ ]:

# Iterate through all team IDs and retrieve box scores for their games
for team_id in team_ids:
    team_games_df = games_df[games_df['TEAM_ID'] == team_id]
    game_ids = team_games_df['GAME_ID'].tolist()
    box_scores = []
    for game_id in game_ids:
        box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        box_score_df = box_score.get_data_frames()[0]
        box_scores.append(box_score_df)
    team_box_score_df = pd.concat(box_scores, axis=0, ignore_index=True)
    team_box_scores.append(team_box_score_df)



In [ ]:
# Concatenate all team box scores into a single dataframe
full_team_box_scores_df = pd.concat(team_box_scores, axis=0, ignore_index=True)

from nba_api.stats.endpoints import BoxScoreTraditionalV2

box_scores = []

for game_id in game_ids:
    box_score = BoxScoreTraditionalV2(game_id=game_id)
    box_score_df = box_score.get_data_frames()[0]
    box_scores.append(box_score_df)


In [ ]:
# Create a new DataFrame for the features
game_data = pd.DataFrame(columns=['GAME_ID', 'TEAM_ID', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'])

# Iterate through each game ID
for game_id in game_ids:
    # Get the box score for the game
    box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
    box_score_df = box_score.get_data_frames()[0]
    
    # Iterate through each team in the game
    for team_id in box_score_df['TEAM_ID'].unique():
        team_box_score_df = box_score_df[box_score_df['TEAM_ID'] == team_id]
        
        # Calculate the features for the team in the game
        fg_pct = team_box_score_df['FG_PCT'].iloc[0]
        fg3_pct = team_box_score_df['FG3_PCT'].iloc[0]
        ft_pct = team_box_score_df['FT_PCT'].iloc[0]
        oreb = team_box_score_df['OREB'].iloc[0]
        dreb = team_box_score_df['DREB'].iloc[0]
        reb = team_box_score_df['REB'].iloc[0]
        ast = team_box_score_df['AST'].iloc[0]
        stl = team_box_score_df['STL'].iloc[0]
        blk = team_box_score_df['BLK'].iloc[0]
        tov = team_box_score_df['TOV'].iloc[0]
        pf = team_box_score_df['PF'].iloc[0]
        plus_minus = team_box_score_df['PLUS_MINUS'].iloc[0]
        
        # Add the features to the game data DataFrame
        game_data = game_data.append({'GAME_ID': game_id, 'TEAM_ID': team_id, 'FG_PCT': fg_pct, 'FG3_PCT': fg3_pct, 'FT_PCT': ft_pct, 'OREB': oreb, 'DREB': dreb, 'REB': reb, 'AST': ast


In [ ]:

# Iterate  game_data = game_data.append({'GAME_ID': game_id, 'TEAM_ID': team_id, 'FG_PCT': fg_pct, 'FG3_PCT': fg3_pct, 'FT_PCT': ft_pct, 'OREB': oreb, 'DREB': dreb, 'REB': reb, 'AST': ast, 'STL': stl, 'BLK': blk, 'TOV': tov, 'PF': pf, 'PLUS_MINUS': plus_minus}, ignore_index=True)
# Add a column to the game_data DataFrame for the winning team
game_data['WINNING_TEAM_ID'] = None

# Iterate through each game ID
for game_id in game_ids:
    # Get the box score for the game
    box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
    box_score_df = box_score.get_data_frames()[0]
    
    # Determine the winning team for the game
    if box_score_df['PLUS_MINUS'].iloc[0] > 0:
        winning_team_id = box_score_df['TEAM_ID'].iloc[0]
    else:
        winning_team_id = box_score_df['TEAM_ID'].iloc[1]
        
    # Update the game_data DataFrame with the winning team ID
    game_data.loc[game_data['GAME_ID'] == game_id, 'WINNING_TEAM_ID'] = winning_team_id



In [ ]:
# Split the game_data DataFrame into training and testing sets
train_data = game_data.sample(frac=0.8, random_state=1)
test_data = game_data.drop(train_data.index)


In [ ]:
# Extract the features and target variables from the training and testing sets
train_features = train_data.drop(['GAME_ID', 'TEAM_ID'], axis=1)
train_targets = train_data['TEAM_ID']
test_features = test_data.drop(['GAME_ID', 'TEAM_ID'], axis=1)
test_targets = test_data['TEAM_ID']


In [ ]:
# Train a machine learning model on the training data
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(train_features, train_targets)

# Test the machine learning model on the testing data
predictions = model.predict(test_features)

# Evaluate the accuracy of the machine learning model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_targets, predictions)
print('Model accuracy:', accuracy)